In [1]:
import pandas as pd
data = pd.read_csv('winequality-red.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [3]:
data['quality'].value_counts()

quality
5    681
6    638
7    199
4     53
8     18
3     10
Name: count, dtype: int64

In [4]:
def simplify_quality(score):
    if score <= 4:
        return 'Low'
    elif score <= 6:
        return 'Medium'
    else:
        return 'High'

data['quality'] = data['quality'].apply(simplify_quality)
data['quality'].value_counts()

quality
Medium    1319
High       217
Low         63
Name: count, dtype: int64

In [5]:
quality_map = {'Low': 0, 'Medium': 1, 'High': 2}
data['quality'] = data['quality'].map(quality_map)

In [6]:
print(data.isnull().sum())

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64


In [7]:
X = data.drop('quality', axis=1)
y = data['quality']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
import xgboost as xgb
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

c:\Users\Jaspreet_workstation\Desktop\Workspace\Wine_Quality_Prediction_Project\wine_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:27:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [10]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Initial Accuracy: {accuracy:.4f}')

Initial Accuracy: 0.8625


In [11]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [100, 200, 300]
}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
best_pred = best_model.predict(X_test)
best_accuracy = accuracy_score(y_test, best_pred)
print(f'Best Accuracy: {best_accuracy:.4f}')
print(f'Best Parameters: {grid_search.best_params_}')

c:\Users\Jaspreet_workstation\Desktop\Workspace\Wine_Quality_Prediction_Project\wine_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:28:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Jaspreet_workstation\Desktop\Workspace\Wine_Quality_Prediction_Project\wine_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:28:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Jaspreet_workstation\Desktop\Workspace\Wine_Quality_Prediction_Project\wine_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:28:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Jaspreet_workstation\Desktop

Best Accuracy: 0.8594
Best Parameters: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}


In [12]:
import joblib
joblib.dump(best_model, 'xgboost_wine_model.pkl')
from sklearn.metrics import classification_report
print(classification_report(y_test, best_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.88      0.96      0.92       262
           2       0.69      0.51      0.59        47

    accuracy                           0.86       320
   macro avg       0.52      0.49      0.50       320
weighted avg       0.82      0.86      0.84       320



c:\Users\Jaspreet_workstation\Desktop\Workspace\Wine_Quality_Prediction_Project\wine_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Jaspreet_workstation\Desktop\Workspace\Wine_Quality_Prediction_Project\wine_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Jaspreet_workstation\Desktop\Workspace\Wine_Quality_Prediction_Project\wine_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with

# Project Summary
# Dataset: Wine Quality (Red) from Kaggle
# Goal: Predict wine quality (Low, Medium, High)
# Steps:
# 1. Loaded winequality-red.csv with pandas
# 2. Preprocessed: Simplified quality into 3 categories, mapped to numbers
# 3. Split into training (80%) and testing (20%) sets
# 4. Trained XGBoost, initial accuracy TBD%
# 5. Tuned with GridSearchCV, best accuracy TBD%
# Tools: Python, pandas, scikit-learn, xgboost